In [63]:
%load_ext autoreload
%autoreload 2

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import os
import warnings 
import glob
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from PIL import Image
import os
from pylab import *
from utils import * 
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

In [115]:
influencers = [
    'mrbeast',
    ]

In [116]:
data = pd.DataFrame()
for influencer in influencers:
    data = pd.concat([data, preprocessing(influencer)])
    data = data.reset_index(drop=True)



In [117]:
images = load_and_prepare_images(data)
likes = data['likes'].values.astype('float32')

In [118]:
enc = OneHotEncoder()
influencer_enc = enc.fit_transform(data[['influencer']])


In [119]:
x_train, x_test, y_train, y_test = train_test_split(images, likes, test_size=0.2, random_state=123)
influencer_train, influencer_test = train_test_split(influencer_enc, test_size=0.2, random_state=123)
x_fit, x_val, y_fit, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=123)
influencer_fit, influencer_val = train_test_split(influencer_train, test_size=0.2, random_state=123)


In [121]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate
from tensorflow.keras.models import Model


# Entrada para las imágenes
image_input = Input(shape=(128, 128, 3))
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D(2, 2)(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(2, 2)(x)
x = Flatten()(x)

# Entrada para la información del influencer
influencer_input = Input(shape=(influencer_enc.shape[1],))
y = Dense(10, activation='relu')(influencer_input)  # Ajusta el tamaño según el número de categorías

# Combinar las entradas
combined = concatenate([x, y])

# Capas totalmente conectadas después de la concatenación
z = Dense(256, activation='relu')(combined)
z = Dense(1)(z)


model = Model(inputs=[image_input, influencer_input], outputs=z)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


model.fit([x_fit, influencer_fit], y_fit, 
          epochs=100, 
          validation_data=([x_val, influencer_val], y_val))

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 622ms/step - loss: 9932186320896.0000 - mae: 2142304.2500 - val_loss: 3521387954176.0000 - val_mae: 1361312.3750
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 9932077268992.0000 - mae: 2142278.5000 - val_loss: 3521271562240.0000 - val_mae: 1361272.2500
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 9931923128320.0000 - mae: 2142241.7500 - val_loss: 3521094352896.0000 - val_mae: 1361210.8750
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 9931685101568.0000 - mae: 2142185.7500 - val_loss: 3520842694656.0000 - val_mae: 1361123.1250
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 9931343265792.0000 - mae: 2142105.7500 - val_loss: 3520495091712.0000 - val_mae: 1361000.8750
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 9930871406592.0000 - mae: 2141995.0000 - val_loss: 3520026378240.0000 - val_mae: 1360836.3750
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 9930231775232.0000 - ma

In [122]:
predictions = model.predict([x_test, influencer_test])
mean_absolute_percentage_error(y_test, predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


1.2915745

In [103]:
predictions

array([[ 839267.7 ],
       [ 909740.56],
       [ 499886.06],
       [ 885880.9 ],
       [1052844.5 ],
       [ 892427.3 ],
       [ 733697.3 ],
       [ 892732.8 ],
       [ 809933.3 ],
       [ 558419.5 ],
       [1252252.9 ],
       [1190082.6 ],
       [ 903920.1 ],
       [ 687176.75],
       [1139819.  ],
       [ 883855.75],
       [1203421.1 ],
       [ 626344.56],
       [1177921.6 ],
       [1091915.6 ],
       [1270213.2 ],
       [ 748908.44],
       [1013196.56],
       [ 693124.94],
       [1222086.2 ],
       [1144292.  ],
       [ 727675.06],
       [1175763.6 ]], dtype=float32)

In [124]:
Application_Credentials = '/Users/aladelca/Downloads/massive-acrobat-421018-1d8b6ce1a11a.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = Application_Credentials

In [128]:
from google.cloud import vision

def detect_objects(path):
    df = pd.DataFrame()
    client = vision.ImageAnnotatorClient()

    with open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.object_localization(image=image)
    objects = []
    scores = []
    for object_ in response.localized_object_annotations:
        objects.append(object_.name)
        scores.append(object_.score)
      
    df['objects'] = objects
    df['scores'] = scores  
    df['path'] = path
    
    return df
def get_features_images(influencer):
    df = pd.DataFrame()
    for i in glob.glob(f'{influencer}/images/*jpg'):
        a = detect_objects(i)
        df = pd.concat([df, a])
    return df

[0.83376145362854]
[0.83376145362854, 0.8136793375015259]
[0.83376145362854, 0.8136793375015259, 0.7688645720481873]
[0.83376145362854, 0.8136793375015259, 0.7688645720481873, 0.7401401996612549]
[0.83376145362854, 0.8136793375015259, 0.7688645720481873, 0.7401401996612549, 0.7254951000213623]
[0.83376145362854, 0.8136793375015259, 0.7688645720481873, 0.7401401996612549, 0.7254951000213623, 0.7003640532493591]
[0.83376145362854, 0.8136793375015259, 0.7688645720481873, 0.7401401996612549, 0.7254951000213623, 0.7003640532493591, 0.6433358788490295]
[0.83376145362854, 0.8136793375015259, 0.7688645720481873, 0.7401401996612549, 0.7254951000213623, 0.7003640532493591, 0.6433358788490295, 0.6041290760040283]
[0.83376145362854, 0.8136793375015259, 0.7688645720481873, 0.7401401996612549, 0.7254951000213623, 0.7003640532493591, 0.6433358788490295, 0.6041290760040283, 0.6025601029396057]
[0.83376145362854, 0.8136793375015259, 0.7688645720481873, 0.7401401996612549, 0.7254951000213623, 0.70036405

In [129]:
a

,objects,scores,path
0,Person,0.833761,mrbeast/images/2023-11-25_17-00-32_UTC_1.jpg
1,Person,0.813679,mrbeast/images/2023-11-25_17-00-32_UTC_1.jpg
2,Top,0.768865,mrbeast/images/2023-11-25_17-00-32_UTC_1.jpg
3,Toy,0.740140,mrbeast/images/2023-11-25_17-00-32_UTC_1.jpg
4,Rugby ball,0.725495,mrbeast/images/2023-11-25_17-00-32_UTC_1.jpg
5,Clothing,0.700364,mrbeast/images/2023-11-25_17-00-32_UTC_1.jpg
6,Ball,0.643336,mrbeast/images/2023-11-25_17-00-32_UTC_1.jpg
7,Pants,0.604129,mrbeast/images/2023-11-25_17-00-32_UTC_1.jpg
8,Clothing,0.602560,mrbeast/images/2023-11-25_17-00-32_UTC_1.jpg
9,Person,0.586830,mrbeast/images/2023-11-25_17-00-32_UTC_1.jpg
